In [20]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import warnings
import time
import sys
import datetime
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.linear_model import BayesianRidge
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
import gc
pd.set_option('display.width',None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_info_columns', 500)
np.random.seed(4950)

In [21]:
train = pd.read_csv("./data/pre_train_clip.csv", index_col=0)
train.drop(columns=[
    'hist_first_year', 'hist_first_quarter', 'hist_first_month',
    'hist_now_year', 'hist_now_quarter', 'hist_now_month',
    
    'hist_installments_sum', 'hist_i_vs_p2p',
    'new_installments_sum', 'new_i_vs_p2p',
    'c_i_diff', 'p2p_i_diff',
    'c_i_diff_vs', 'p2p_i_diff_vs',

#     'hist_sleep', 'hist_sleep_vs_count', 'hist_sleep_vs_p2p',
#     'new_sleep', 'new_sleep_vs_count', 'new_sleep_vs_p2p',
#     'c_sleep_diff', 'p2p_sleep_diff', 
    
#     'c_p2p_diff_vs', 'c_sleep_diff_vs', 'c_p_diff_vs', 
#     'p2p_count_diff_vs', 'p2p_sleep_diff_vs', 'p2p_p_diff_vs', 
                   ], inplace=True)

test = pd.read_csv("./data/pre_test_clip.csv", index_col=0)
test.drop(columns=[
    'hist_first_year', 'hist_first_quarter', 'hist_first_month',
    'hist_now_year', 'hist_now_quarter', 'hist_now_month',
    
    'hist_installments_sum', 'hist_i_vs_p2p',
    'new_installments_sum', 'new_i_vs_p2p',
    'c_i_diff', 'p2p_i_diff',
    'c_i_diff_vs', 'p2p_i_diff_vs',

#     'hist_sleep', 'hist_sleep_vs_count', 'hist_sleep_vs_p2p',
#     'new_sleep', 'new_sleep_vs_count', 'new_sleep_vs_p2p',
#     'c_sleep_diff', 'p2p_sleep_diff', 
    
#     'c_p2p_diff_vs', 'c_sleep_diff_vs', 'c_p_diff_vs', 
#     'p2p_count_diff_vs', 'p2p_sleep_diff_vs', 'p2p_p_diff_vs', 
                   ], inplace=True)

cats = [
        'feature_1', 'feature_2', 'feature_3', 
#         'hist_first_year', 
#         'hist_first_quarter', 
#         'hist_first_month',
        'hist_re_year', 
        'hist_re_quarter', 
#         'hist_re_month',
#         'hist_now_year', 
#         'hist_now_quarter', 
#         'hist_now_month',
]

train[cats] = train[cats].fillna(-1, )
test[cats] = test[cats].fillna(-1, )

In [22]:
# train['hist_re_year'].unique()

In [23]:
train['hist_re_quarter'].unique()

array([1, 3, 4, 2])

In [24]:
train['hist_re_month'].unique()

array([ 2,  1,  7, 12, 10,  5,  3, 11,  9,  8,  4,  6])

In [25]:
test[test['card_id']=='C_ID_c27b4f80f7']

,first_active_month,card_id,feature_1,feature_2,feature_3,authorized_flag_mean,hist_transactions_count,hist_is_month_start_mean,hist_weekend_mean,hist_category_1_mean,hist_category_2_nunique,hist_category_3_nunique,hist_state_id_nunique,hist_city_id_nunique,hist_subsector_id_nunique,hist_merchant_category_id_nunique,hist_merchant_id_nunique,hist_quarter_nunique,hist_month_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_day_nunique,hist_hour_nunique,hist_a2p_mean,hist_a2p_median,hist_a2p_max,hist_a2p_min,hist_a2p_std,hist_p2r_mean,hist_p2r_median,hist_p2r_max,hist_p2r_min,hist_p2r_std,hist_p2now_mean,hist_p2now_median,hist_p2now_max,hist_p2now_min,hist_p2now_std,hist_month_lag_mean,hist_month_lag_median,hist_month_lag_max,hist_month_lag_min,hist_month_lag_std,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_median,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_mean,hist_installments_median,hist_installments_max,hist_installments_min,hist_installments_std,hist_p_vs_m_mean,hist_p_vs_m_median,hist_p_vs_m_max,hist_p_vs_m_min,hist_p_vs_m_std,hist_p_vs_i_mean,hist_p_vs_i_median,hist_p_vs_i_max,hist_p_vs_i_min,hist_p_vs_i_std,hist_purchase_date_max,hist_purchase_date_min,hist_category_2_p_mean_mean,hist_category_2_p_mean_median,hist_category_2_p_mean_max,hist_category_2_p_mean_min,hist_category_2_p_mean_std,hist_category_3_p_mean_mean,hist_category_3_p_mean_median,hist_category_3_p_mean_max,hist_category_3_p_mean_min,hist_category_3_p_mean_std,hist_state_id_p_mean_mean,hist_state_id_p_mean_median,hist_state_id_p_mean_max,hist_state_id_p_mean_min,hist_state_id_p_mean_std,hist_city_id_p_mean_mean,hist_city_id_p_mean_median,hist_city_id_p_mean_max,hist_city_id_p_mean_min,hist_city_id_p_mean_std,hist_subsector_id_p_mean_mean,hist_subsector_id_p_mean_median,hist_subsector_id_p_mean_max,hist_subsector_id_p_mean_min,hist_subsector_id_p_mean_std,hist_merchant_category_id_p_mean_mean,hist_merchant_category_id_p_mean_median,hist_merchant_category_id_p_mean_max,hist_merchant_category_id_p_mean_min,hist_merchant_category_id_p_mean_std,hist_merchant_id_p_mean_mean,hist_merchant_id_p_mean_median,hist_merchant_id_p_mean_max,hist_merchant_id_p_mean_min,hist_merchant_id_p_mean_std,hist_quarter_p_mean_mean,hist_quarter_p_mean_median,hist_quarter_p_mean_max,hist_quarter_p_mean_min,hist_quarter_p_mean_std,hist_month_p_mean_mean,hist_month_p_mean_median,hist_month_p_mean_max,hist_month_p_mean_min,hist_month_p_mean_std,hist_weekofyear_p_mean_mean,hist_weekofyear_p_mean_median,hist_weekofyear_p_mean_max,hist_weekofyear_p_mean_min,hist_weekofyear_p_mean_std,hist_dayofweek_p_mean_mean,hist_dayofweek_p_mean_median,hist_dayofweek_p_mean_max,hist_dayofweek_p_mean_min,hist_dayofweek_p_mean_std,hist_day_p_mean_mean,hist_day_p_mean_median,hist_day_p_mean_max,hist_day_p_mean_min,hist_day_p_mean_std,hist_hour_p_mean_mean,hist_hour_p_mean_median,hist_hour_p_mean_max,hist_hour_p_mean_min,hist_hour_p_mean_std,hist_re_year,hist_re_quarter,hist_re_month,hist_a2r,hist_r2now,hist_a2now,hist_p2p,hist_sleep,hist_p2p_vs_count,hist_sleep_vs_count,hist_count_vs_p2p,hist_sleep_vs_p2p,hist_p_vs_p2p,new_transactions_count,new_is_month_start_mean,new_weekend_mean,new_category_1_mean,new_category_2_nunique,new_category_3_nunique,new_state_id_nunique,new_city_id_nunique,new_subsector_id_nunique,new_merchant_category_id_nunique,new_merchant_id_nunique,new_quarter_nunique,new_month_nunique,new_weekofyear_nunique,new_dayofweek_nunique,new_day_nunique,new_hour_nunique,new_a2p_mean,new_a2p_median,new_a2p_max,new_a2p_min,new_a2p_std,new_p2r_mean,new_p2r_median,new_p2r_max,new_p2r_min,new_p2r_std,new_p2now_mean,new_p2now_median,new_p2now_max,new_p2now_min,new_p2now_std,new_month_lag_mean,new_month_lag_median,new_month_lag_max,new_month_lag_min,new_month_lag_std,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_median,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_i

In [26]:
data = pd.concat([train, test])    
data.reset_index(drop=True, inplace=True)

for feat in cats:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
#     data[feat][data[feat]>=0] = lbe.fit_transform(data[feat][data[feat]>=0])

In [27]:
train[cats] = data[0:201917][cats]

In [28]:
tmp = data[201917:][cats].reset_index(drop=True)
test[cats] = tmp

In [29]:
for c in cats:
    print(train[c].unique())
    print(test[c].unique())
    print(data[c].unique())

[4 3 1 0 2]
[2 1 4 0 3]
[4 3 1 0 2]
[1 0 2]
[2 0 1]
[1 0 2]
[1 0]
[1 0]
[1 0]
[1 0]
[0 1]
[1 0]
[0 2 3 1]
[3 0 2 1]
[0 2 3 1]


In [30]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,authorized_flag_mean,hist_transactions_count,hist_is_month_start_mean,hist_weekend_mean,hist_category_1_mean,hist_category_2_nunique,hist_category_3_nunique,hist_state_id_nunique,hist_city_id_nunique,hist_subsector_id_nunique,hist_merchant_category_id_nunique,hist_merchant_id_nunique,hist_quarter_nunique,hist_month_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_day_nunique,hist_hour_nunique,hist_a2p_mean,hist_a2p_median,hist_a2p_max,hist_a2p_min,hist_a2p_std,hist_p2r_mean,hist_p2r_median,hist_p2r_max,hist_p2r_min,hist_p2r_std,hist_p2now_mean,hist_p2now_median,hist_p2now_max,hist_p2now_min,hist_p2now_std,hist_month_lag_mean,hist_month_lag_median,hist_month_lag_max,hist_month_lag_min,hist_month_lag_std,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_median,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_mean,hist_installments_median,hist_installments_max,hist_installments_min,hist_installments_std,hist_p_vs_m_mean,hist_p_vs_m_median,hist_p_vs_m_max,hist_p_vs_m_min,hist_p_vs_m_std,hist_p_vs_i_mean,hist_p_vs_i_median,hist_p_vs_i_max,hist_p_vs_i_min,hist_p_vs_i_std,hist_purchase_date_max,hist_purchase_date_min,hist_category_2_p_mean_mean,hist_category_2_p_mean_median,hist_category_2_p_mean_max,hist_category_2_p_mean_min,hist_category_2_p_mean_std,hist_category_3_p_mean_mean,hist_category_3_p_mean_median,hist_category_3_p_mean_max,hist_category_3_p_mean_min,hist_category_3_p_mean_std,hist_state_id_p_mean_mean,hist_state_id_p_mean_median,hist_state_id_p_mean_max,hist_state_id_p_mean_min,hist_state_id_p_mean_std,hist_city_id_p_mean_mean,hist_city_id_p_mean_median,hist_city_id_p_mean_max,hist_city_id_p_mean_min,hist_city_id_p_mean_std,hist_subsector_id_p_mean_mean,hist_subsector_id_p_mean_median,hist_subsector_id_p_mean_max,hist_subsector_id_p_mean_min,hist_subsector_id_p_mean_std,hist_merchant_category_id_p_mean_mean,hist_merchant_category_id_p_mean_median,hist_merchant_category_id_p_mean_max,hist_merchant_category_id_p_mean_min,hist_merchant_category_id_p_mean_std,hist_merchant_id_p_mean_mean,hist_merchant_id_p_mean_median,hist_merchant_id_p_mean_max,hist_merchant_id_p_mean_min,hist_merchant_id_p_mean_std,hist_quarter_p_mean_mean,hist_quarter_p_mean_median,hist_quarter_p_mean_max,hist_quarter_p_mean_min,hist_quarter_p_mean_std,hist_month_p_mean_mean,hist_month_p_mean_median,hist_month_p_mean_max,hist_month_p_mean_min,hist_month_p_mean_std,hist_weekofyear_p_mean_mean,hist_weekofyear_p_mean_median,hist_weekofyear_p_mean_max,hist_weekofyear_p_mean_min,hist_weekofyear_p_mean_std,hist_dayofweek_p_mean_mean,hist_dayofweek_p_mean_median,hist_dayofweek_p_mean_max,hist_dayofweek_p_mean_min,hist_dayofweek_p_mean_std,hist_day_p_mean_mean,hist_day_p_mean_median,hist_day_p_mean_max,hist_day_p_mean_min,hist_day_p_mean_std,hist_hour_p_mean_mean,hist_hour_p_mean_median,hist_hour_p_mean_max,hist_hour_p_mean_min,hist_hour_p_mean_std,hist_re_year,hist_re_quarter,hist_re_month,hist_a2r,hist_r2now,hist_a2now,hist_p2p,hist_sleep,hist_p2p_vs_count,hist_sleep_vs_count,hist_count_vs_p2p,hist_sleep_vs_p2p,hist_p_vs_p2p,new_transactions_count,new_is_month_start_mean,new_weekend_mean,new_category_1_mean,new_category_2_nunique,new_category_3_nunique,new_state_id_nunique,new_city_id_nunique,new_subsector_id_nunique,new_merchant_category_id_nunique,new_merchant_id_nunique,new_quarter_nunique,new_month_nunique,new_weekofyear_nunique,new_dayofweek_nunique,new_day_nunique,new_hour_nunique,new_a2p_mean,new_a2p_median,new_a2p_max,new_a2p_min,new_a2p_std,new_p2r_mean,new_p2r_median,new_p2r_max,new_p2r_min,new_p2r_std,new_p2now_mean,new_p2now_median,new_p2now_max,new_p2now_min,new_p2now_std,new_month_lag_mean,new_month_lag_median,new_month_lag_max,new_month_lag_min,new_month_lag_std,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_median,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_st

In [31]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,authorized_flag_mean,hist_transactions_count,hist_is_month_start_mean,hist_weekend_mean,hist_category_1_mean,hist_category_2_nunique,hist_category_3_nunique,hist_state_id_nunique,hist_city_id_nunique,hist_subsector_id_nunique,hist_merchant_category_id_nunique,hist_merchant_id_nunique,hist_quarter_nunique,hist_month_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_day_nunique,hist_hour_nunique,hist_a2p_mean,hist_a2p_median,hist_a2p_max,hist_a2p_min,hist_a2p_std,hist_p2r_mean,hist_p2r_median,hist_p2r_max,hist_p2r_min,hist_p2r_std,hist_p2now_mean,hist_p2now_median,hist_p2now_max,hist_p2now_min,hist_p2now_std,hist_month_lag_mean,hist_month_lag_median,hist_month_lag_max,hist_month_lag_min,hist_month_lag_std,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_median,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_mean,hist_installments_median,hist_installments_max,hist_installments_min,hist_installments_std,hist_p_vs_m_mean,hist_p_vs_m_median,hist_p_vs_m_max,hist_p_vs_m_min,hist_p_vs_m_std,hist_p_vs_i_mean,hist_p_vs_i_median,hist_p_vs_i_max,hist_p_vs_i_min,hist_p_vs_i_std,hist_purchase_date_max,hist_purchase_date_min,hist_category_2_p_mean_mean,hist_category_2_p_mean_median,hist_category_2_p_mean_max,hist_category_2_p_mean_min,hist_category_2_p_mean_std,hist_category_3_p_mean_mean,hist_category_3_p_mean_median,hist_category_3_p_mean_max,hist_category_3_p_mean_min,hist_category_3_p_mean_std,hist_state_id_p_mean_mean,hist_state_id_p_mean_median,hist_state_id_p_mean_max,hist_state_id_p_mean_min,hist_state_id_p_mean_std,hist_city_id_p_mean_mean,hist_city_id_p_mean_median,hist_city_id_p_mean_max,hist_city_id_p_mean_min,hist_city_id_p_mean_std,hist_subsector_id_p_mean_mean,hist_subsector_id_p_mean_median,hist_subsector_id_p_mean_max,hist_subsector_id_p_mean_min,hist_subsector_id_p_mean_std,hist_merchant_category_id_p_mean_mean,hist_merchant_category_id_p_mean_median,hist_merchant_category_id_p_mean_max,hist_merchant_category_id_p_mean_min,hist_merchant_category_id_p_mean_std,hist_merchant_id_p_mean_mean,hist_merchant_id_p_mean_median,hist_merchant_id_p_mean_max,hist_merchant_id_p_mean_min,hist_merchant_id_p_mean_std,hist_quarter_p_mean_mean,hist_quarter_p_mean_median,hist_quarter_p_mean_max,hist_quarter_p_mean_min,hist_quarter_p_mean_std,hist_month_p_mean_mean,hist_month_p_mean_median,hist_month_p_mean_max,hist_month_p_mean_min,hist_month_p_mean_std,hist_weekofyear_p_mean_mean,hist_weekofyear_p_mean_median,hist_weekofyear_p_mean_max,hist_weekofyear_p_mean_min,hist_weekofyear_p_mean_std,hist_dayofweek_p_mean_mean,hist_dayofweek_p_mean_median,hist_dayofweek_p_mean_max,hist_dayofweek_p_mean_min,hist_dayofweek_p_mean_std,hist_day_p_mean_mean,hist_day_p_mean_median,hist_day_p_mean_max,hist_day_p_mean_min,hist_day_p_mean_std,hist_hour_p_mean_mean,hist_hour_p_mean_median,hist_hour_p_mean_max,hist_hour_p_mean_min,hist_hour_p_mean_std,hist_re_year,hist_re_quarter,hist_re_month,hist_a2r,hist_r2now,hist_a2now,hist_p2p,hist_sleep,hist_p2p_vs_count,hist_sleep_vs_count,hist_count_vs_p2p,hist_sleep_vs_p2p,hist_p_vs_p2p,new_transactions_count,new_is_month_start_mean,new_weekend_mean,new_category_1_mean,new_category_2_nunique,new_category_3_nunique,new_state_id_nunique,new_city_id_nunique,new_subsector_id_nunique,new_merchant_category_id_nunique,new_merchant_id_nunique,new_quarter_nunique,new_month_nunique,new_weekofyear_nunique,new_dayofweek_nunique,new_day_nunique,new_hour_nunique,new_a2p_mean,new_a2p_median,new_a2p_max,new_a2p_min,new_a2p_std,new_p2r_mean,new_p2r_median,new_p2r_max,new_p2r_min,new_p2r_std,new_p2now_mean,new_p2now_median,new_p2now_max,new_p2now_min,new_p2now_std,new_month_lag_mean,new_month_lag_median,new_month_lag_max,new_month_lag_min,new_month_lag_std,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_median,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_i

In [32]:
fold_target = train['target']
fold_target = (fold_target<-20).astype(int)

In [33]:
features = ['feature_1', 'feature_2', 'authorized_flag_mean', 'hist_transactions_count', 'hist_is_month_start_mean', 'hist_category_1_mean', 'hist_merchant_category_id_nunique', 'hist_merchant_id_nunique', 'hist_month_nunique', 'hist_weekofyear_nunique', 'hist_day_nunique', 'hist_a2p_mean', 'hist_a2p_median', 'hist_a2p_min', 'hist_p2now_max', 'hist_purchase_amount_sum', 'hist_installments_mean', 'hist_installments_max', 'hist_installments_min', 'hist_p_vs_m_mean', 'hist_p_vs_m_std', 'hist_p_vs_i_median', 'hist_purchase_date_max', 'hist_purchase_date_min', 'hist_category_2_p_mean_std', 'hist_category_3_p_mean_mean', 'hist_category_3_p_mean_max', 'hist_state_id_p_mean_std', 'hist_subsector_id_p_mean_min', 'hist_merchant_id_p_mean_median', 'hist_merchant_id_p_mean_max', 'hist_merchant_id_p_mean_min', 'hist_hour_p_mean_min', 'hist_re_year', 'hist_re_quarter', 'hist_re_month', 'hist_a2r', 'hist_r2now', 'hist_a2now', 'hist_p2p', 'hist_count_vs_p2p', 'hist_sleep_vs_p2p', 'hist_p_vs_p2p', 'new_transactions_count', 'new_category_1_mean', 'new_subsector_id_nunique', 'new_merchant_category_id_nunique', 'new_merchant_id_nunique', 'new_month_nunique', 'new_a2p_mean', 'new_a2p_std', 'new_p2r_mean', 'new_p2r_max', 'new_p2r_min', 'new_p2r_std', 'new_p2now_mean', 'new_p2now_max', 'new_p2now_min', 'new_p2now_std', 'new_month_lag_std', 'new_purchase_amount_max', 'new_purchase_amount_min', 'new_installments_min', 'new_p_vs_m_max', 'new_p_vs_m_min', 'new_p_vs_i_mean', 'new_purchase_date_max', 'new_purchase_date_min', 'new_category_3_p_mean_mean', 'new_quarter_p_mean_mean', 'new_month_p_mean_max', 'new_dayofweek_p_mean_max', 'new_hour_p_mean_min', 'new_sleep_vs_count', 'new_count_vs_p2p', 'c_sleep_diff', 'c_p_diff', 'p2p_p_diff', 'c_p2p_diff_vs', 'c_sleep_diff_vs', 'p2p_count_diff_vs']

In [34]:
cat_feats = ['feature_1', 'feature_2', 'hist_re_year', 'hist_re_quarter']

In [35]:
print(features)
print(cat_feats)

['feature_1', 'feature_2', 'authorized_flag_mean', 'hist_transactions_count', 'hist_is_month_start_mean', 'hist_category_1_mean', 'hist_merchant_category_id_nunique', 'hist_merchant_id_nunique', 'hist_month_nunique', 'hist_weekofyear_nunique', 'hist_day_nunique', 'hist_a2p_mean', 'hist_a2p_median', 'hist_a2p_min', 'hist_p2now_max', 'hist_purchase_amount_sum', 'hist_installments_mean', 'hist_installments_max', 'hist_installments_min', 'hist_p_vs_m_mean', 'hist_p_vs_m_std', 'hist_p_vs_i_median', 'hist_purchase_date_max', 'hist_purchase_date_min', 'hist_category_2_p_mean_std', 'hist_category_3_p_mean_mean', 'hist_category_3_p_mean_max', 'hist_state_id_p_mean_std', 'hist_subsector_id_p_mean_min', 'hist_merchant_id_p_mean_median', 'hist_merchant_id_p_mean_max', 'hist_merchant_id_p_mean_min', 'hist_hour_p_mean_min', 'hist_re_year', 'hist_re_quarter', 'hist_re_month', 'hist_a2r', 'hist_r2now', 'hist_a2now', 'hist_p2p', 'hist_count_vs_p2p', 'hist_sleep_vs_p2p', 'hist_p_vs_p2p', 'new_transactio

In [36]:
len(features)

81

In [37]:
# test = pd.read_csv("./data/pre_test_super.csv", index_col=0)
# test.drop(columns=['hist_reference_date', 'new_reference_date'], inplace=True)

In [38]:
param = {
         'objective':'regression',
         "metric": 'rmse',
         "boosting": "gbdt",
         'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'max_depth': -1,
         'learning_rate': 0.01,
         "lambda_l1": 0.1,
         "feature_fraction": 0.9,
         "bagging_fraction": 0.9 ,
         "bagging_freq": 1,
   
         "random_state": 1024,
         "verbosity": -1,
}

n_fold = 5

# param ={
#         'task': 'train',
#         'boosting': 'goss',
#         'objective': 'regression',
#         'metric': 'rmse',
#         'learning_rate': 0.01,
#         'subsample': 0.9855232997390695,
#         'max_depth': 7,
#         'top_rate': 0.9064148448434349,
#         'num_leaves': 63,
#         'min_child_weight': 41.9612869171337,
#         'other_rate': 0.0721768246018207,
#         'reg_alpha': 9.677537745007898,
#         'colsample_bytree': 0.5665320670155495,
#         'min_split_gain': 9.820197773625843,
#         'reg_lambda': 8.2532317400459,
#         'min_data_in_leaf': 21,
#         'verbose': -1,
#         'seed':int(2**n_fold),
#         'bagging_seed':int(2**n_fold),
#         'drop_seed':int(2**n_fold)
#         }

In [39]:
# from sklearn.model_selection import RepeatedKFold
# folds = RepeatedKFold(n_splits=5, n_repeats=2, random_state=2333)
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=4590)
oof_train = np.zeros((len(train),1))
oof_test = np.zeros(len(test))
oof_test_skf = np.zeros((5,len(test),1))
start = time.time()
feature_importance_df = pd.DataFrame()

target = train['target']

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, fold_target)):
    print("fold n°{}".format(fold_+1))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx], categorical_feature=cat_feats)
    val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx], categorical_feature=cat_feats)
#     trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx])
#     val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data],
                    verbose_eval=100, early_stopping_rounds = 200)
    oof_train[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration).reshape(-1,1)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    oof_test_skf[fold_,:]= clf.predict(test[features], num_iteration=clf.best_iteration).reshape(-1,1)
#     oof_test += clf.predict(test[features], num_iteration=clf.best_iteration) / 10
    oof_test += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

fold n°1
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.65345	valid_1's rmse: 3.71435
[200]	training's rmse: 3.57243	valid_1's rmse: 3.67123
[300]	training's rmse: 3.52303	valid_1's rmse: 3.6551
[400]	training's rmse: 3.4858	valid_1's rmse: 3.64632
[500]	training's rmse: 3.45693	valid_1's rmse: 3.64103
[600]	training's rmse: 3.43135	valid_1's rmse: 3.63786
[700]	training's rmse: 3.40932	valid_1's rmse: 3.6353
[800]	training's rmse: 3.38962	valid_1's rmse: 3.63329
[900]	training's rmse: 3.36983	valid_1's rmse: 3.63207
[1000]	training's rmse: 3.35064	valid_1's rmse: 3.63115
[1100]	training's rmse: 3.33343	valid_1's rmse: 3.63089
[1200]	training's rmse: 3.31643	valid_1's rmse: 3.63063
[1300]	training's rmse: 3.3007	valid_1's rmse: 3.63012
[1400]	training's rmse: 3.28628	valid_1's rmse: 3.6299
[1500]	training's rmse: 3.27144	valid_1's rmse: 3.63052
Early stopping, best iteration is:
[1370]	training's rmse: 3.2908	valid_1's rmse: 3.62967
fold n°2
Tra

In [40]:
print("CV score: {:<8.5f}".format(mean_squared_error(oof_train, target)**0.5))

CV score: 3.63686 


In [41]:

mean_loss=np.sqrt(mean_squared_error(oof_train.reshape(-1), target))
print(mean_loss)
sub_df = pd.DataFrame({"card_id":test["card_id"].values})
sub_df["target"] = oof_test
sub_df.to_csv("sub/submission_%s.csv"%mean_loss, index=False)

train_prob=pd.DataFrame(oof_train)
train_prob.columns=['class1']
train_prob.to_csv("oof/train_prob_%s.csv"%mean_loss,index=False)

test_prob=pd.DataFrame(oof_test)
test_prob.columns=['class1']
test_prob.to_csv("oof/test_prob_%s.csv"%mean_loss,index=False)

np.save("train_y",target)

3.636857606814238
